In [1]:
import pandas as pd
import json
from sklearn.cluster import KMeans
import reverse_geocoder as rg
import csv
import os
import pgeocode
from geopy.geocoders import Nominatim

In [2]:
def process_input(folderpath, filename, delimiter = '|', keep_columns = [], key_column = ''):    
    with open(folderpath + filename, "r", encoding = 'utf-8-sig') as csvfile:
        dict_reader = csv.DictReader(csvfile, delimiter = delimiter)

        data = {}
        for row in dict_reader:
            d = {}
            for k in list(row.keys()):
                if k in keep_columns:
                    d[k] = row[k].strip()
            
            data[row[key_column].strip()] = d
    return data


In [3]:
INPUT_DATA_PATH = './files/uniteller/'

all_data = {}
data_updates = {}
missing = {}

print("Processing transactions...")
for f in os.listdir(INPUT_DATA_PATH):
    if f.endswith('.csv') and 'FraudReport_' in f:
        
        data = process_input(INPUT_DATA_PATH, f, delimiter = '|', 
            keep_columns = ['txIdentifier', 'txOriginCountry', 'txAgentState', 'paymentCountry', 'paymentAmount', 'txCreationDateLocal'], 
            key_column = 'txIdentifier')
        all_data.update(data)

for f in os.listdir(INPUT_DATA_PATH):
    if f.endswith('.csv') and 'FraudReport-' in f:
        print("processing updates", f, '...')
        data = process_input(INPUT_DATA_PATH, f, delimiter = ',', 
            keep_columns = ['txIdentifier', 'txOriginCountry', 'txAgentState', 'paymentCountry', 'paymentAmount', 'txCreationDateLocal', 'txAmount(USD)'], 
            key_column = 'txIdentifier')
        data_updates.update(data)

print(f'loaded {len(all_data)} transactions and {len(data_updates)} updates')
for d in all_data:
    if d in data_updates:
        all_data[d].update(data_updates[d])
    else:
        missing[d] = all_data[d]

print(f'Missing {len(missing)} out of {len(all_data)}')
# txOriginCountry, paymentCountry, paymentAmount, txCreationDateLocal

Processing transactions...
processing updates FraudReport-JanMar2023.csv ...
processing updates FraudReport-JulSep2023.csv ...
processing updates FraudReport-OctDec2023.csv ...
processing updates FraudReport-AprJun2023.csv ...
loaded 1906687 transactions and 1850921 updates
Missing 76781 out of 1906687


In [4]:
def split_date(date):
    s = date.split(' ')[0]
    s = ''.join(list(reversed(s.split('/'))))
    return s

In [5]:
updated_data = {k: v for k, v in all_data.items() if k not in missing}
sorted_by_date = sorted(updated_data, key = lambda x: split_date(updated_data[x]['txCreationDateLocal']))

GEOCODE_CACHE = {}


In [6]:

out_data = {}
out_data_by_date = {}
i = 0

geolocator = Nominatim(user_agent="fnunigeo")

for k in sorted_by_date:
    data = {}
    data['amount'] = updated_data[k]['txAmount(USD)']
    data['order_date'] = updated_data[k]['txCreationDateLocal'].split(' ')[0]
    data['from_country'] = updated_data[k]['txOriginCountry']
    data['from_state'] = updated_data[k]['txAgentState']
    data['to_country'] = updated_data[k]['paymentCountry']

    if len(out_data_by_date.get(i, [])) == 0:
        out_data_by_date[i] = [data]
    elif out_data_by_date[i][0]['order_date'] == data['order_date']:
        out_data_by_date[i].append(data)
    else:
        i += 1
        if i > 31:
            break
        out_data_by_date[i] = [data]

# Aggregating data by same from and to countries
for k in out_data_by_date:
    data = out_data_by_date[k]
    data_by_countries = {}

    out_data[k] = []

    for d in data:
        key = f"{d['from_state']}_{d['to_country']}"
        if key not in data_by_countries:
            data_by_countries[key] = {
                'amount': round(float(d['amount']), 2),
                'count': 1
            }
        else:
            data_by_countries[key]['amount'] += round(float(d['amount']), 2)
            data_by_countries[key]['count'] += 1

    for key in data_by_countries:
        d = data_by_countries[key]
        from_country, to_country = key.split('_')

        if from_country not in GEOCODE_CACHE:
            GEOCODE_CACHE[from_country] = geolocator.geocode(from_country)

        if to_country not in GEOCODE_CACHE:
            GEOCODE_CACHE[to_country] = geolocator.geocode(to_country)
        
        out_data[k].append({
            'from_country': from_country,
            'to_country': to_country,
            'total_amount': d['amount'],
            'total_count': d['count'],
            'order_date': data[0]['order_date'],
            'from_lat': GEOCODE_CACHE[from_country].latitude,
            'from_lon': GEOCODE_CACHE[from_country].longitude,
            'to_lat': GEOCODE_CACHE[to_country].latitude,
            'to_lon': GEOCODE_CACHE[to_country].longitude
        })

with open('files/transactions_uniteller.json', 'w') as f:
    json.dump(out_data, f, indent = 0)

In [7]:
updated_data[sorted_by_date[-1]]

{'txIdentifier': '702531449644',
 'paymentAmount': '27088',
 'paymentCountry': 'MEXICO',
 'txAgentState': 'CALIFORNIA',
 'txOriginCountry': 'USA',
 'txCreationDateLocal': '27/12/2023 23:58',
 'txAmount(USD)': '1600'}

In [8]:
split_date(updated_data[sorted_by_date[-1]]['txCreationDateLocal'])

'20231227'